The goal is to create a RL agent that is capable of finding the pid gains for a quadcopter control system

In [2]:
!pip install gym pybullet numpy stable-baselines3

In [ ]:
!pip install gym

In [5]:
!pip install 'shimmy>=2.0'

In [1]:
import pybullet as p
import pybullet_data
import gym
import numpy as np
from gym import spaces

class DroneEnv(gym.Env):
    def __init__(self):
        super(DroneEnv, self).__init__()

        # Connect using DIRECT mode (no GUI)
        self.physicsClient = p.connect(p.GUI)  

        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.8)

        # Load drone model
        self.drone = p.loadURDF("samurai.urdf", basePosition=[0, 0, 1])

        # Define action and observation spaces
        self.action_space = spaces.Box(low=np.array([0, 0, 0]), high=np.array([10, 10, 10]), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=np.float32)

        self.target_position = np.array([5, 5, 5])
        self.state = np.zeros(6)

    def step(self, action):
        """Apply PID gains and simulate drone dynamics."""
        Kp, Ki, Kd = action
        pos, orn = p.getBasePositionAndOrientation(self.drone)
        vel, ang_vel = p.getBaseVelocity(self.drone)

        error = self.target_position - np.array(pos)
        force = Kp * error - Kd * np.array(vel)
        p.applyExternalForce(self.drone, -1, force, [0, 0, 0], p.WORLD_FRAME)

        self.state = np.hstack((pos, ang_vel))
        reward = -np.linalg.norm(error)
        done = np.linalg.norm(error) < 0.1

        return self.state, reward, done, {}

    def reset(self):
        """Reset drone to initial position."""
        p.resetBasePositionAndOrientation(self.drone, [0, 0, 1], [0, 0, 0, 1])
        self.state = np.zeros(6)
        return self.state
    
    def render(self, mode="human"):
        """Refresh the PyBullet GUI."""
        p.stepSimulation()  # Ensure simulation updates in GUI


    def close(self):
        p.disconnect()


pybullet build time: Mar 18 2025 08:09:30


In [ ]:
# Train PPO Agent
from stable_baselines3 import PPO

env = DroneEnv()
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=100000)
model.save("drone_pid_ppo")

In [ ]:
DroneEnv().close()

Version = 4.1 Metal - 89.3
Vendor = Apple
Renderer = Apple M3
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


/Users/ortol/miniforge3/lib/python3.12/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed


: 

In [ ]:
from stable_baselines3 import PPO
import time

# Load the trained model
model = PPO.load("drone_pid_ppo")

# Create environment
env = DroneEnv()

# Run simulation using the trained model
obs = env.reset()
for _ in range(1000):  # Simulate for 1000 steps
    action, _ = model.predict(obs)  # Use the trained model to get actions
    obs, reward, done, _ = env.step(action)  # Apply the action
    env.render()  # Show drone movement
    time.sleep(0.05)
    if done:
        obs = env.reset()



Version = 4.1 Metal - 89.3
Vendor = Apple
Renderer = Apple M3
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


/Users/ortol/miniforge3/lib/python3.12/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


: 